In [1]:
!pip install yfinance
!pip install transformers

In [1]:
import yfinance as yf
import pandas as pd
import os
import json
import datetime
from datetime import date,timedelta
import warnings
import http.client, urllib.parse
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline
# warnings.filterwarnings("ignore")


2023-11-05 22:41:09.219620: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 22:41:09.252943: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-05 22:41:09.252984: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-05 22:41:09.253016: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-05 22:41:09.259723: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

In [2]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

### New Section

In [3]:
def create_news_sentiment_dataset(news_sentiments):
    title_description= news_sentiments['Description'].values.tolist()
    news_label_and_scores = classifier(title_description)
    labels=[pred['label'] for pred in news_label_and_scores]
    scores=[pred['score'] for pred in news_label_and_scores ]
    news_sentiments = news_sentiments.assign(labels=labels)
    news_sentiments = news_sentiments.assign(scores=scores)
    return news_sentiments

In [5]:
df = pd.read_csv('scrapped_input/reliance.csv',index_col=None)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
reliance_sentiment = create_news_sentiment_dataset(df)

NameError: name 'create_news_sentiment_dataset' is not defined

In [6]:
reliance_sentiment = reliance_sentiment.drop('Stocks', axis=1)

In [7]:
reliance_sentiment

,Date,Headlines,Description,labels,scores
0,2023-11-04 10:54:18,Ambani’s Reliance Industries Considers Record ...,Indian tycoon Mukesh Ambani’s Reliance Industr...,neutral,0.918048
1,2023-10-31 09:17:55,Mukesh Ambani receives death threat for third ...,Reliance Industries chairperson has been recei...,negative,0.910378
2,2023-10-31 05:01:28,Mukesh Ambani receives third threat email with...,"Mukesh Ambani, Chairman of Reliance Industries...",negative,0.878922
3,2023-10-30 10:11:10,Jio World Plaza: Largest Luxury Mall Of India ...,India is about to gain a new luxury shopping d...,positive,0.828098
4,2023-10-30 04:34:14,Reliance Industries Q2 Review: Analysts See St...,Analysts see Reliance Industries Ltd.'s full-y...,positive,0.949535
...,...,...,...,...,...
117,2023-07-20 03:40:48,Jio Financial Demerger: Reliance Industries Sp...,The F&O contracts of Reliance Industries Ltd. ...,neutral,0.949252
118,2023-07-20 00:57:00,"Day Trading Guide for July 20, 2023: Intraday ...",Day Trading Guide gives you the key intraday s...,neutral,0.896284
119,2023-07-19 02:59:58,Today is the last date to buy Reliance shares ...,Reliance Industries must be bought today to ge...,neutral,0.950439
120,2023-07-19 02:36:35,"Stocks that will see action today: July 19, 2023",Reliance Industries shares turn ex-date on Jul...,neutral,0.642565


In [8]:
reliance_sentiment.to_csv('sentiment_outputs/reliance_sentiment.csv', index=False)